In [ ]:
import yfinance as yf
import datetime

# Define the stock symbol and time range
stock_symbol = 'AAPL'
start_date = '2015-01-01'
end_date = '2024-10-01'

# Fetch the stock data using yfinance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Display the first few rows of the dataset
print(stock_data.head())


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt

# Define the stock symbol and time range
stock_symbol = 'AAPL'
start_date = '2015-01-01'
end_date = '2024-10-01'

# Fetch the stock data using yfinance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Plot the closing price over time
plt.figure(figsize=(10, 6))
plt.plot(stock_data.index, stock_data['Close'], label='Closing Price', color='blue')
plt.title('Apple (AAPL) Stock Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Closing Price (USD)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import pandas as pd

# Create the 50-day and 200-day Simple Moving Averages (SMA)
stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()
stock_data['SMA_200'] = stock_data['Close'].rolling(window=200).mean()

# Create the 50-day Exponential Moving Average (EMA)
stock_data['EMA_50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()

# Create the Relative Strength Index (RSI) - 14-day
delta = stock_data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
stock_data['RSI_14'] = 100 - (100 / (1 + rs))

# Create the MACD (12-day EMA - 26-day EMA)
stock_data['EMA_12'] = stock_data['Close'].ewm(span=12, adjust=False).mean()
stock_data['EMA_26'] = stock_data['Close'].ewm(span=26, adjust=False).mean()
stock_data['MACD'] = stock_data['EMA_12'] - stock_data['EMA_26']

# Create Volatility (Standard Deviation over a 30-day window)
stock_data['Volatility_30'] = stock_data['Close'].rolling(window=30).std()

# Drop any NaN values that may have been created
stock_data = stock_data.dropna()

# Display the new columns with the features
stock_data[['SMA_50', 'SMA_200', 'EMA_50', 'RSI_14', 'MACD', 'Volatility_30']].head()


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Data Preprocessing

# Use only the 'Close' price for prediction
data = stock_data['Close'].values.reshape(-1, 1)

# Scale the data to be between 0 and 1 (important for LSTM)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences for LSTM (e.g., 60 days of data to predict the next day)
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Reshape X for LSTM input [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Step 2: Model Creation

# Initialize the LSTM model
model = Sequential()

# Add LSTM layers with Dropout to prevent overfitting
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Add a Dense layer for output
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 3: Model Training

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Step 4: Prediction & Evaluation

# Prepare the test data (you can split the original data into train/test)
test_data = stock_data['Close'][-sequence_length:].values.reshape(-1, 1)
scaled_test_data = scaler.transform(test_data)

# Reshape the test data to fit the LSTM input shape
X_test = []
for i in range(sequence_length, len(scaled_test_data)):
    X_test.append(scaled_test_data[i-sequence_length:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Predict using the trained model
predicted_stock_price = model.predict(X_test)

# Inverse transform the scaled predictions to get actual values
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

# Step 5: Visualization of Predictions vs Actual Prices

plt.figure(figsize=(10,6))
plt.plot(stock_data.index[-len(predicted_stock_price):], test_data, color='blue', label='Actual Stock Price')
plt.plot(stock_data.index[-len(predicted_stock_price):], predicted_stock_price, color='red', label='Predicted Stock Price')
plt.title('Stock Price Prediction using LSTM')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.show()


In [ ]:
# Split the data into training and testing sets
train_size = int(len(stock_data) * 0.8)  # 80% of data for training
train_data = stock_data['Close'][:train_size].values.reshape(-1, 1)
test_data = stock_data['Close'][train_size - sequence_length:].values.reshape(-1, 1)

# Scale the training and testing data
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

# Prepare the training data sequences
X_train = []
y_train = []

for i in range(sequence_length, len(scaled_train_data)):
    X_train.append(scaled_train_data[i-sequence_length:i, 0])
    y_train.append(scaled_train_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Prepare the testing data sequences
X_test = []
y_test = test_data[sequence_length:]

for i in range(sequence_length, len(scaled_test_data)):
    X_test.append(scaled_test_data[i-sequence_length:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Model Training (keep as it was)
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict using the trained model
predicted_stock_price = model.predict(X_test)

# Inverse transform the scaled predictions and actual prices
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

# Visualization of Predictions vs Actual Prices
plt.figure(figsize=(10,6))
plt.plot(stock_data.index[train_size:], test_data[sequence_length:], color='blue', label='Actual Stock Price')
plt.plot(stock_data.index[train_size:], predicted_stock_price, color='red', label='Predicted Stock Price')
plt.title('Stock Price Prediction using LSTM')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.show()
